In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,2021-03-13T17:00:00,ITA,19,Sicilia,38.115697,13.362357,684,99,783,13087,...,NaN,5.0,NaN,NaN,160194.0,0.0,1849759.0,904079.0,ITG,ITG1
8060,2021-03-13T17:00:00,ITA,9,Toscana,43.769231,11.255889,1332,217,1549,22180,...,NaN,16.0,NaN,NaN,170574.0,1103.0,2652952.0,369718.0,ITI,ITI1
8061,2021-03-13T17:00:00,ITA,10,Umbria,43.106758,12.388247,391,80,471,5681,...,"Si fa presente che 7 dei ricoveri NON UTI, non...",7.0,NaN,NaN,47707.0,0.0,737818.0,151295.0,ITI,ITI2
8062,2021-03-13T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,15,2,17,255,...,NaN,0.0,NaN,NaN,8271.0,0.0,76952.0,6136.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,2021-03-13T17:00:00,19,Sicilia,684,99,783,13087,13870,74,650,...,160194,2753838,1191491.0,5.0,160194.0,0.0,1849759.0,904079.0,ITG,ITG1
8060,2021-03-13T17:00:00,9,Toscana,1332,217,1549,22180,23729,535,1326,...,171677,3022670,1597260.0,16.0,170574.0,1103.0,2652952.0,369718.0,ITI,ITI1
8061,2021-03-13T17:00:00,10,Umbria,391,80,471,5681,6152,-209,196,...,47707,889113,324742.0,7.0,47707.0,0.0,737818.0,151295.0,ITI,ITI2
8062,2021-03-13T17:00:00,2,Valle d'Aosta,15,2,17,255,272,25,31,...,8271,83088,46415.0,0.0,8271.0,0.0,76952.0,6136.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-13', '2021-03-12')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-13T17:00:00,1,Piemonte,2790,251,3041,25636,28677,1038,2159,...,271949,2883525,1366697.0,25.0,262860.0,9089.0,2199408.0,684117.0,ITC,ITC1
1,2021-03-13T17:00:00,2,Valle d'Aosta,15,2,17,255,272,25,31,...,8271,83088,46415.0,0.0,8271.0,0.0,76952.0,6136.0,ITC,ITC2
2,2021-03-13T17:00:00,3,Lombardia,6068,694,6762,87731,94493,1768,5809,...,662704,7288380,3296269.0,54.0,643597.0,19107.0,6688718.0,599662.0,ITC,ITC4
3,2021-03-13T17:00:00,5,Veneto,1111,170,1281,32781,34062,1512,2682,...,352727,5521174,1458362.0,30.0,346024.0,6703.0,4230649.0,1290525.0,ITH,ITH3
4,2021-03-13T17:00:00,6,Friuli Venezia Giulia,504,65,569,13606,14175,434,953,...,85543,1460244,522935.0,11.0,77277.0,8266.0,1328053.0,132191.0,ITH,ITH4
5,2021-03-13T17:00:00,7,Liguria,512,64,576,5653,6229,38,307,...,82323,1120022,473756.0,4.0,82323.0,0.0,981821.0,138201.0,ITC,ITC3
6,2021-03-13T17:00:00,8,Emilia-Romagna,3254,354,3608,60879,64487,1870,2950,...,296324,4317841,1592539.0,33.0,296144.0,180.0,3670207.0,647634.0,ITH,ITH5
7,2021-03-13T17:00:00,9,Toscana,1332,217,1549,22180,23729,535,1326,...,171677,3022670,1597260.0,16.0,170574.0,1103.0,2652952.0,369718.0,ITI,ITI1
8,2021-03-13T17:00:00,10,Umbria,391,80,471,5681,6152,-209,196,...,47707,889113,324742.0,7.0,47707.0,0.0,737818.0,151295.0,ITI,ITI2
9,2021-03-13T17:00:00,11,Marche,695,118,813,10250,11063,205,1000,...,77559,912707,557484.0,15.0,77559.0,0.0,829299.0,83408.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-12T17:00:00,1,Piemonte,2684,247,2931,24708,27639,1634,2929,...,269790,2851854,1356275.0,16.0,261093.0,8697.0,2188926.0,662928.0,ITC,ITC1
1,2021-03-12T17:00:00,2,Valle d'Aosta,12,2,14,233,247,17,28,...,8240,82395,46155.0,0.0,8240.0,0.0,76776.0,5619.0,ITC,ITC2
2,2021-03-12T17:00:00,3,Lombardia,5909,667,6576,86149,92725,4057,6262,...,656895,7229002,3280523.0,39.0,638816.0,18079.0,6648598.0,580404.0,ITC,ITC4
3,2021-03-12T17:00:00,5,Veneto,1106,154,1260,31290,32550,876,1932,...,350045,5469375,1450815.0,22.0,343453.0,6592.0,4209207.0,1260168.0,ITH,ITH3
4,2021-03-12T17:00:00,6,Friuli Venezia Giulia,498,65,563,13178,13741,567,971,...,84590,1449956,518821.0,7.0,76648.0,7942.0,1320966.0,128990.0,ITH,ITH4
5,2021-03-12T17:00:00,7,Liguria,512,64,576,5615,6191,131,406,...,82016,1113258,471127.0,3.0,82016.0,0.0,977708.0,135550.0,ITC,ITC3
6,2021-03-12T17:00:00,8,Emilia-Romagna,3197,350,3547,59070,62617,1951,3477,...,293400,4278613,1585015.0,23.0,293223.0,177.0,3646581.0,632032.0,ITH,ITH5
7,2021-03-12T17:00:00,9,Toscana,1292,218,1510,21684,23194,297,1304,...,170351,2999126,1585470.0,21.0,169287.0,1064.0,2638802.0,360324.0,ITI,ITI1
8,2021-03-12T17:00:00,10,Umbria,403,79,482,5879,6361,-236,219,...,47511,882935,323934.0,5.0,47511.0,0.0,734364.0,148571.0,ITI,ITI2
9,2021-03-12T17:00:00,11,Marche,673,111,784,10074,10858,26,874,...,76559,906059,553703.0,11.0,76559.0,0.0,823598.0,82461.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-13T17:00:00,1,Piemonte,2790,251,3041,25636,28677,1038,2159,...,2883525,1366697.0,25.0,262860.0,9089.0,2199408.0,684117.0,ITC,ITC1,4.341375e+06
1,2021-03-13T17:00:00,2,Valle d'Aosta,15,2,17,255,272,25,31,...,83088,46415.0,0.0,8271.0,0.0,76952.0,6136.0,ITC,ITC2,1.255010e+05
2,2021-03-13T17:00:00,3,Lombardia,6068,694,6762,87731,94493,1768,5809,...,7288380,3296269.0,54.0,643597.0,19107.0,6688718.0,599662.0,ITC,ITC4,1.010397e+07
3,2021-03-13T17:00:00,5,Veneto,1111,170,1281,32781,34062,1512,2682,...,5521174,1458362.0,30.0,346024.0,6703.0,4230649.0,1290525.0,ITH,ITH3,4.907704e+06
4,2021-03-13T17:00:00,6,Friuli Venezia Giulia,504,65,569,13606,14175,434,953,...,1460244,522935.0,11.0,77277.0,8266.0,1328053.0,132191.0,ITH,ITH4,1.211357e+06
5,2021-03-13T17:00:00,7,Liguria,512,64,576,5653,6229,38,307,...,1120022,473756.0,4.0,82323.0,0.0,981821.0,138201.0,ITC,ITC3,1.543127e+06
6,2021-03-13T17:00:00,8,Emilia-Romagna,3254,354,3608,60879,64487,1870,2950,...,4317841,1592539.0,33.0,296144.0,180.0,3670207.0,647634.0,ITH,ITH5,4.467118e+06
7,2021-03-13T17:00:00,9,Toscana,1332,217,1549,22180,23729,535,1326,...,3022670,1597260.0,16.0,170574.0,1103.0,2652952.0,369718.0,ITI,ITI1,3.722729e+06
8,2021-03-13T17:00:00,10,Umbria,391,80,471,5681,6152,-209,196,...,889113,324742.0,7.0,47707.0,0.0,737818.0,151295.0,ITI,ITI2,8.802850e+05
9,2021-03-13T17:00:00,11,Marche,695,118,813,10250,11063,205,1000,...,912707,557484.0,15.0,77559.0,0.0,829299.0,83408.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-13T17:00:00,1,Piemonte,2790,251,3041,25636,28677,1038,2159,...,684117.0,ITC,ITC1,4.341375e+06,Piemonte,2684,5824,247,628,99
1,2021-03-13T17:00:00,2,Valle d'Aosta,15,2,17,255,272,25,31,...,6136.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,12,227,2,20,12
2,2021-03-13T17:00:00,3,Lombardia,6068,694,6762,87731,94493,1768,5809,...,599662.0,ITC,ITC4,1.010397e+07,Lombardia,5909,12259,667,1416,114
3,2021-03-13T17:00:00,5,Veneto,1111,170,1281,32781,34062,1512,2682,...,1290525.0,ITH,ITH3,4.907704e+06,Veneto,1106,6000,154,1000,0
4,2021-03-13T17:00:00,6,Friuli Venezia Giulia,504,65,569,13606,14175,434,953,...,132191.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,498,1277,65,175,0
5,2021-03-13T17:00:00,7,Liguria,512,64,576,5653,6229,38,307,...,138201.0,ITC,ITC3,1.543127e+06,Liguria,512,1794,64,224,10
6,2021-03-13T17:00:00,8,Emilia-Romagna,3254,354,3608,60879,64487,1870,2950,...,647634.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,3197,6441,350,760,0
7,2021-03-13T17:00:00,9,Toscana,1332,217,1549,22180,23729,535,1326,...,369718.0,ITI,ITI1,3.722729e+06,Toscana,1292,5033,218,599,15
8,2021-03-13T17:00:00,10,Umbria,391,80,471,5681,6152,-209,196,...,151295.0,ITI,ITI2,8.802850e+05,Umbria,403,852,79,139,2
9,2021-03-13T17:00:00,11,Marche,695,118,813,10250,11063,205,1000,...,83408.0,ITI,ITI3,1.518400e+06,Marche,673,1176,111,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-13T17:00:00,3,Lombardia,6068,694,6762,87731,94493,1768,5809,...,49.01,1582,-453,59378,15746.0,66,3975,9.78,0.05749,6.56
1,2021-03-13T17:00:00,8,Emilia-Romagna,3254,354,3608,60879,64487,1870,2950,...,46.58,1809,-527,39228,7524.0,31,1023,7.52,0.06604,6.63
2,2021-03-13T17:00:00,15,Campania,1547,150,1697,94435,96132,1228,2940,...,24.19,1182,296,27263,16529.0,23,1689,10.78,0.05081,5.21
3,2021-03-13T17:00:00,5,Veneto,1111,170,1281,32781,34062,1512,2682,...,17.00,1491,750,51799,7547.0,24,1146,5.18,0.05465,7.19
4,2021-03-13T17:00:00,1,Piemonte,2790,251,3041,25636,28677,1038,2159,...,39.97,928,-770,31671,10422.0,28,1093,6.82,0.04973,6.26
5,2021-03-13T17:00:00,12,Lazio,2172,278,2450,39055,41505,1191,1998,...,29.48,1139,241,42167,23103.0,19,788,4.74,0.03406,4.33
6,2021-03-13T17:00:00,16,Puglia,1442,175,1617,36750,38367,1061,1700,...,30.76,1027,-74,10322,5860.0,23,616,16.47,0.04241,4.09
7,2021-03-13T17:00:00,9,Toscana,1332,217,1549,22180,23729,535,1326,...,36.23,496,22,23544,11790.0,20,771,5.63,0.03562,4.61
8,2021-03-13T17:00:00,11,Marche,695,118,813,10250,11063,205,1000,...,50.64,176,126,6648,3781.0,11,784,15.04,0.06586,5.11
9,2021-03-13T17:00:00,6,Friuli Venezia Giulia,504,65,569,13606,14175,434,953,...,37.14,428,-18,10288,4114.0,15,504,9.26,0.07867,7.06


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,80,59,139,57.55
1,Marche,118,115,233,50.64
2,P.A. Trento,45,45,90,50.00
3,Lombardia,694,722,1416,49.01
4,Emilia-Romagna,354,406,760,46.58
5,Molise,18,21,39,46.15
6,Abruzzo,94,120,214,43.93
7,Piemonte,251,377,628,39.97
8,Friuli Venezia Giulia,65,110,175,37.14
9,Toscana,217,382,599,36.23


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))